In [1]:
require(tidyverse)


Loading required package: tidyverse
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0       ✔ purrr   0.3.1  
✔ tibble  2.0.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
require(data.table)


Loading required package: data.table

Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

The following object is masked from ‘package:purrr’:

    transpose



In [41]:
read_keep <- function(name){
    keep_dir <- '/scratch/PI/mrivas/users/erflynn/sex_div_gwas/phefiles'
    df <- fread(file.path(keep_dir, paste0(name, '.keep')))
    colnames(df) <- c('FID')
    df %>% mutate(IID = FID, keep = name)
}


In [3]:
t_med <- fread('/scratch/PI/mrivas/users/erflynn/sex_div_gwas/data/t_related.txt')


In [14]:
t_med %>% dim() %>% print()

t_med %>% gather("med", "val", -IID) %>% filter(val > 0) %>% select(IID) %>% unique() %>% dim() %>% print()

[1] 2766    7
[1] 2766    1


In [70]:
read_phe <- function(){
    phe_dir <- '/oak/stanford/groups/mrivas/projects/biomarkers/covariate_corrected'
    phe_suffix <- 'phenotypes/residual/Testosterone.phe'
    
    bind_rows(lapply(c(
        'outputExtendedBMIreducedMaleWhiteBritish',
        'outputExtendedBMIreducedFemaleWhiteBritish'
    ), function(x){
        fread(file.path(phe_dir, x, phe_suffix))%>%
        filter(! IID %in% (t_med %>% select(IID) %>% pull())) %>%
        drop_na()
    }))
}


In [71]:
t_phe <- read_phe()

In [72]:
t_phe %>% dim()


[1] 365682      3

In [73]:
df <- bind_rows(lapply(c('pre_meno', 'post_meno', 'onesex', 'zerosex'), function(x){
    read_keep(x) %>% 
    inner_join(t_phe, by=c('FID', 'IID'))
}))


In [78]:
df %>% select(FID) %>% unique() %>% dim()

[1] 365682      1

In [74]:
df %>% count(keep)


keep,n
onesex,182552
post_meno,98816
pre_meno,41291
zerosex,183130


In [83]:
phe_data_dir <- '/oak/stanford/groups/mrivas/users/ytanigaw/repos/rivas-lab/sex-div-analysis/snpnet/phe_data'
for (k in c('pre_meno', 'post_meno', 'onesex', 'zerosex')){
    df %>% filter(keep == k) %>% select(-keep) %>% 
    fwrite(file.path(phe_data_dir, paste0(k, '.phe')), sep='\t', )
}
